In [ ]:
%pip install opencv-python-headless numpy torch shapely ultralytics redis flask

In [ ]:
import cv2
import numpy as np
import time
import torch
from ultralytics import YOLO
from shapely.geometry import Polygon, Point
import redis
import json
import subprocess

In [ ]:
r = redis.Redis(host='localhost', port=6379, db=0)

In [ ]:
class ParkingSpaceMonitor:
    def __init__(self, video_source, yolo_model_path, parking_id, parking_spaces_coords, crop_points=None, 
                 check_interval=10, consecutive_checks=3, confidence_threshold=0.5):
        """Инициализация системы мониторинга парковочных мест"""
        self.video_source = video_source
        self.yolo_model_path = yolo_model_path
        self.parking_id = parking_id
        self.parking_spaces_coords = parking_spaces_coords
        self.crop_points = crop_points
        self.check_interval = check_interval
        self.consecutive_checks = consecutive_checks
        self.confidence_threshold = confidence_threshold

        # Инициализация модели YOLO
        self.model = YOLO(yolo_model_path)

        # Только один класс - Vehicle с ID 0
        self.vehicle_classes = [0]
        self.class_names = {0: 'Vehicle'}

        # Инициализация парковочных мест
        self.parking_spaces = self._initialize_parking_spaces(parking_spaces_coords)

        # Команда ffmpeg для обработки HLS
        self.ffmpeg_command = [
            'ffmpeg', '-i', self.video_source, '-loglevel', 'quiet', '-f', 'image2pipe',
            '-pix_fmt', 'bgr24', '-vcodec', 'rawvideo', '-'
        ]

    def _initialize_parking_spaces(self, parking_spaces_coords):
        """Инициализация парковочных мест"""
        parking_spaces = {}
        for space_id, coords in parking_spaces_coords.items():
            parking_spaces[space_id] = {
                'polygon': Polygon(coords),
                'status': 'free',
                'history': []
            }
        return parking_spaces
    
    def get_crop_size(self):
        """Определяет размер выходного изображения по crop_points"""
        if not self.crop_points or len(self.crop_points) != 4:
            return None
        width = int(np.linalg.norm(np.array(self.crop_points[1]) - np.array(self.crop_points[0])))
        height = int(np.linalg.norm(np.array(self.crop_points[3]) - np.array(self.crop_points[0])))
        return (width, height)

    def crop_frame(self, frame):
        """Обрезает изображение по заданным точкам"""
        if not self.crop_points:
            return frame
        width, height = self.get_crop_size()
        src_points = np.array(self.crop_points, dtype=np.float32)
        dst_points = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype=np.float32)
        matrix = cv2.getPerspectiveTransform(src_points, dst_points)
        return cv2.warpPerspective(frame, matrix, (width, height))

    
    def detect_objects(self, frame):
        """Обнаруживает объекты на кадре с помощью модели YOLO"""
        results = self.model(frame)
        result = results[0]
        vehicles = []
        for box in result.boxes:
            cls_id = int(box.cls.item())
            confidence = float(box.conf.item())
            if cls_id in self.vehicle_classes and confidence >= self.confidence_threshold:
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                vehicles.append({
                    'center': Point((x1 + x2) // 2, (y1 + y2) // 2),
                    'bbox': (x1, y1, x2, y2),
                    'class': self.class_names.get(cls_id, 'Vehicle'),
                    'confidence': confidence
                })
        return vehicles

    def check_parking_spaces(self, vehicles, occupation_threshold=0.4):
        """Проверяет занятость парковочных мест"""
        current_status = {}
        for space_id, space_info in self.parking_spaces.items():
            space_polygon = space_info['polygon']
            occupied_area = sum(
                space_polygon.intersection(Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)])).area
                for vehicle in vehicles
                for x1, y1, x2, y2 in [vehicle['bbox']]
                if space_polygon.intersects(Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)]))
            )
            occupation_percentage = occupied_area / space_polygon.area if space_polygon.area > 0 else 0
            current_status[space_id] = 'occupied' if occupation_percentage >= occupation_threshold else 'free'
        return current_status

    def update_parking_status(self, current_status):
        """Обновляет статусы парковочных мест и возвращает список изменений"""
        status_changes = []
        for space_id, current in current_status.items():
            history = self.parking_spaces[space_id]['history']
            history.append(current)
            if len(history) > self.consecutive_checks:
                history.pop(0)
            if len(history) == self.consecutive_checks and all(status == current for status in history):
                if current != self.parking_spaces[space_id]['status']:
                    status_changes.append((space_id, self.parking_spaces[space_id]['status'], current))
                    self.parking_spaces[space_id]['status'] = current
        return status_changes

    def visualize(self, frame, parking_spaces):
        """Отображает на кадре статус парковочных мест"""
        for space_id, space_info in parking_spaces.items():
            coords = np.array(space_info['polygon'].exterior.coords, np.int32)
            color = (0, 255, 0) if space_info['status'] == 'free' else (0, 0, 255)
            cv2.polylines(frame, [coords], True, color, 2)
            cv2.putText(frame, f"ID: {space_id}", coords[0], cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        return frame

    def process_frame(self, frame):
        """Обработка одного кадра"""
        # Обрезаем кадр, если необходимо
        frame = self.crop_frame(frame)

        # Обнаружение объектов
        vehicles = self.detect_objects(frame)

        # Проверка занятости мест
        current_status = self.check_parking_spaces(vehicles)

        # Обновление статусов
        status_changes = self.update_parking_status(current_status)

        # Вывод изменений статусов
        for space_id, old_status, new_status in status_changes:
            print(f"Парковочное место {space_id} изменило статус с '{old_status}' на '{new_status}'")
            key = f"parking:{self.parking_id}:space:{space_id}"
            r.hset(key, mapping={
                "status": new_status,
                "timestamp": int(time.time())
            })
            r.publish(
                f"parking:{self.parking_id}:updates",
                json.dumps({
                    "space_id": space_id,
                    "status": new_status,
                    "timestamp": int(time.time())
                })
            )

        # Выделение машин на кадре
        for vehicle in vehicles:
            x1, y1, x2, y2 = vehicle['bbox']
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 165, 0), 2)  # Оранжевый цвет
            label = f"{vehicle['class']}: {vehicle['confidence']:.2f}"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 165, 0), 2)

        # Визуализация и возврат кадра
        return self.visualize(frame, self.parking_spaces)

    def run_hls_stream(self):
        """Обработка HLS-потока"""
        # 1. Сначала определяем исходный размер кадра (например, 1920x1080)
        src_width, src_height = 1920, 1080  # Укажите реальный размер потока!
        frame_bytes = src_width * src_height * 3

        pipe = subprocess.Popen(self.ffmpeg_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, bufsize=10**8)

        fps = 25
        frame_interval = int(fps * self.check_interval)
        frame_count = 0

        while True:
            try:
                raw_frame = pipe.stdout.read(frame_bytes)
                if len(raw_frame) != frame_bytes:
                    continue
                # 2. Читаем исходный кадр
                frame = np.frombuffer(raw_frame, dtype=np.uint8).reshape((src_height, src_width, 3)).copy()

                # 3. Только теперь делаем crop_frame (она сама обрежет до нужного размера)
                if frame_count % frame_interval == 0:
                    processed_frame = self.process_frame(frame)
                    cv2.imshow('Parking Monitor', processed_frame)
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break

                frame_count += 1
            except Exception as e:
                print(f"Ошибка обработки кадра: {str(e)}")
                continue
        pipe.terminate()
        cv2.destroyAllWindows()

    def run_video_file(self):
        """Обработка видеофайла"""
        cap = cv2.VideoCapture(self.video_source)
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_interval = int(fps * self.check_interval)
        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            # Обрабатываем только каждый N-й кадр
            if frame_count % frame_interval == 0:
                processed_frame = self.process_frame(frame)
                cv2.imshow('Parking Monitor', processed_frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            frame_count += 1
        cap.release()
        cv2.destroyAllWindows()

    def run(self):
        """Запуск обработки в зависимости от типа источника"""
        if self.video_source.startswith('http'):
            self.run_hls_stream()
        else:
            self.run_video_file()

In [ ]:
# Пример использования:
    # Определяем координаты парковочных мест
    # Формат: {id_места: [(x1, y1), (x2, y2), (x3, y3), (x4, y4)], ...}
parking_spaces = {
    1: [(348, 73), (402, 77), (423, 119), (359, 111)],
    2: [(293, 60), (346, 70), (355, 108), (285, 100)],
    3: [(232, 49), (289, 59), (283, 107), (207, 96)],
    4: [(176, 48), (235, 52), (206, 95), (140, 89)],
    5: [(127, 43), (177, 48), (139, 88), (86, 84)],
}

crop_points = [(1340, 560), (1860, 560), (1860, 960), (1340, 960)]
    
    # Инициализируем и запускаем мониторинг
monitor = ParkingSpaceMonitor(
    video_source='https://cameras.inetcom.ru/hls/camera12_4.m3u8',  # Путь к видеофайлу или HLS-поток https://cameras.inetcom.ru/hls/camera12_4.m3u8
    yolo_model_path='car-75e-11n.pt',
    parking_id='0',
    parking_spaces_coords=parking_spaces,
    crop_points=crop_points,  
    check_interval=2,
    consecutive_checks=2,
    confidence_threshold=0.5
)

monitor.run()
